In [1]:
# Intro to One class SVM: http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/
# How to use Support Vector Machines for One-Class Classification?: https://analyticsindiamag.com/how-to-use-support-vector-machines-for-one-class-classification/
# Advantages & Disadvantages of SVM: https://dhirajkumarblog.medium.com/top-4-advantages-and-disadvantages-of-support-vector-machine-or-svm-a3c06a2b107

# Papers #
# Support Vector Method for Novelty Detection (Schölkopf - distance maximization from origin to hyperplane):https://proceedings.neurips.cc/paper/1999/file/8725fb777f25776ffa9076e44fcfd776-Paper.pdf
# Support Vector Data Description (Tax and Duin - spherical approach):https://link.springer.com/content/pdf/10.1023/B:MACH.0000008084.60811.49.pdf

# Used in this JN: https://www.analyticsvidhya.com/blog/2022/06/one-class-classification-using-support-vector-machines/

# Clustering Methods (Extra): https://estreuselito.github.io/Deus_ex_machina/Mini-Project/Master.html
# Novelty and Outlier detection (extra): https://scikit-learn.org/stable/modules/outlier_detection.html#overview-of-outlier-detection-methods

### Microsoft Azure doc: https://docs.microsoft.com/en-us/previous-versions/azure/machine-learning/studio-module-reference/one-class-support-vector-machine?redirectedfrom=MSDN

Support vector machines (SVMs) are supervised learning models that analyze data and recognize patterns, and that can be used for both classification and regression tasks.

Typically, the SVM algorithm is given a set of training examples labeled as belonging to one of two classes. An SVM model is based on dividing the training sample points into separate categories by as wide a gap as possible, while penalizing training samples that fall on the wrong side of the gap. The SVM model then makes predictions by assigning points to one side of the gap or the other.

Sometimes oversampling is used to replicate the existing samples so that you can create a two-class model, but it is impossible to predict all the new patterns of fraud or system faults from limited examples. Moreover, collection of even limited examples can be expensive.

Therefore, in one-class SVM, the support vector model is trained on data that has only one class, which is the “normal” class. It infers the properties of normal cases and from these properties can predict which examples are unlike the normal examples. This is useful for anomaly detection because the scarcity of training examples is what defines anomalies: that is, typically there are very few examples of the network intrusion, fraud, or other anomalous behavior.

### One-class SVM vs. exemplar SVM: https://stats.stackexchange.com/questions/63028/one-class-svm-vs-exemplar-svm

Let's start with the "classic" support vector machines. These learn to discriminate between two categories. You collect some examples of category A, some of category B and pass them both to the SVM training algorithm, which finds the line/plane/hyperplane that best separates A from B. This works--and it often works quite well--when you want to distinguish between well-defined and mutually exclusive classes: men vs. women, the letters of the alphabet, and so on.

However, suppose you want to identify "A"s instead. You could treat this as a classification problem: How do I distinguish "A"s from "not-A"s. It is fairly easy to gather up a training set consisting of pictures of dogs, but what should go into your training set of not-dogs? Since there are an infinite number of things that are not dogs, you might have a difficult time constructing a comprehensive and yet representative training set of all non-canine things. Instead, you might consider using a one-class classifier. The traditional, two-class classifier finds a (hyper)plane that separates A from B. The one-class SVM instead finds the line/plane/hyperplane that separates all of the in-class points (the "A"s) from origin; it is essentially a two-class SVM where the origin is the only member of the second class (finding the maximum margin from the origin is pretty similar to finding the smallest sphere that contains all As, which might make more conceptual sense).

The Ensemble SVM "system" is actually a collection of many two-class SVM "subunits". Each subunit is trained using a single positive example for one class and an enormous collection of negative examples for the other. Thus, instead of discriminating dogs vs. not-dog examples (standard two-class SVM), or dogs vs. origin (one-class SVM), each subunit discriminates between specific dog (e.g., "Rex") and many not-dog examples. Individual subunit SVMs are trained for each example of the positive class, so you would have one SVM for Rex, another for Fido, yet another for your neighbour’s dog that barks at 6am, and so on. The outputs of these subunit SVMs are calibrated and combined to determine whether a dog, not just one of the specific exemplars, appears in the test data. I guess you could also think of the individual subnits as somewhat like one-class SVMs, where the coordinate space is shifted so that the single positive example lies at the origin.

In summary, the key differences are:

## TRAINING DATA

Two class SVM: Positive and negative examples.

One class SVM: Positive examples only.

Ensemble SVM "system": Positive and negative examples. Each subunit is trained on a single positive example and many negative examples.

## NUMBER OF MACHINES

Two class SVM: one.

One class SVM: one.

Ensemble SVM "system": many (one subunit machine per positive example).

## EXAMPLES PER CLASS (PER MACHINE)

Two class SVM: many/many.

One class SVM: many/one (fixed at the origin).

Ensemble SVM "system": many/many.

Ensemble SVM "subunit": one/many.

## POST-PROCESSING

Two class SVM: Not necessary.

One class SVM: Not necessary.

Ensemble SVM: Needed to fuse each SVM's output into a class-level prediction.

Postscript: You had asked what they mean by "[other approaches] require mapping the exemplars into a common feature space over which a similarity kernel can be computed." I think they mean that a traditional two-class SVM operates under the assumption that all members of class are somehow similar, and so you want to find a kernel that places great danes and dachsunds near each other, but far away from everything else. By contrast, the ensemble SVM system sidesteps this by calling something a dog if it's sufficiently great dane-like OR dachsund-like OR poodle-like, without worrying about the relationship between those exemplars.

In [2]:
# One class SVM 10 data points.
from sklearn.svm import OneClassSVM

X = [[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]]

y = [[-1],[1],[-2],[2],[-3]]

one_svm = OneClassSVM(gamma='auto', nu=0.01).fit(X)

# gamma is used to set the kernel function for forming the hypersphere to learn and
# differienciate samples and the hyperparameter nu is tuned to approximate the ratio
# of outliers

one_svm.predict(y)

# estimator predict method is used to classify the data points between classes 1, -1
# based on the training data

array([-1,  1, -1,  1, -1])

In [3]:
one_svm.score_samples(y)
# score_samples method is used to access the scoring function of the estimator and the 
# contamination parameter is used to set the threshold for classification 

array([2.47729192e-04, 1.67496679e-02, 1.66310989e-06, 1.70740848e-02,
       1.51581080e-09])

In [4]:
one_svm.decision_function(y)
# decision_function returns the value such that the negative values represents the
# sample to be outlier or out of training distribution

array([-0.01639928,  0.00010266, -0.01664534,  0.00042708, -0.016647  ])

### More Data

In [5]:
import time
import numpy as np

start = time.time()

np.random.seed(7)
# Generating random numbers between 400 and 600 to train one-class SVM. This is the positive class
randtrain= np.random.randint(400,601,100000)
mas_li=[]
for x in randtrain:
    mas_li.append(x)
    
train_li=[]
for y in mas_li:
    temp_li=[y]
    train_li.append(temp_li)

In [6]:
# Generating random numbers between 300 and 700 to test one-class SVM. Positive and negative classes.
randtest = np.random.randint(300,700,100)

int_li=[]
for x in randtest:
    int_li.append(x)
    
test_list=[]
for y in int_li:
    temp_li=[y]
    test_list.append(temp_li)
    
one_svm = OneClassSVM(gamma='auto').fit(train_li)
pred_result = one_svm.predict(test_list)

In [7]:
correct_result=[]
for x in range(len(test_list)):
    if test_list[x][0] < 400:
        correct_result.append(-1)
       
    elif test_list[x][0] > 600:
        correct_result.append(-1)
    else:
        correct_result.append(1)
    

In [13]:
y=0
for x in range(len(pred_result)):
    if pred_result[x] == correct_result[x]:
        y+=1
print('Correctly classified: ',y/len(pred_result))

Correctly classified:  0.75


In [9]:
time_spend = time.time() - start
print('time spent: ', time_spend)

time spent:  252.3668270111084
